In [ ]:
#run this in colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q drive/MyDrive/data/cropped.zip

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.layers import LSTM

In [ ]:
data = pd.read_csv('drive/MyDrive/data/data.csv')

def preprocess_image(image_path, target_size=(28, 28)):
    if pd.isna(image_path):
        return None
    image_path = image_path.split('/')[-1]
    img = load_img(f"cropped/{image_path}", target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=-1)
    img = img.astype('float32') / 255.0
    return img

# Filter out NaN values before processing
valid_image_paths = data['image'].dropna()

images = []
for path in tqdm(valid_image_paths):
    processed_image = preprocess_image(path)
    if processed_image is not None:
        images.append(processed_image)
images = np.array(images)

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data.loc[valid_image_paths.index, 'phone'])
labels = to_categorical(labels)



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

X_train = X_train.reshape((X_train.shape[0], 28, 28 * 3))
X_test = X_test.reshape((X_test.shape[0], 28, 28 * 3))

unique_phones = np.unique(data['phone'].dropna().astype(str))

model = models.Sequential([
    LSTM(128, input_shape=(28, 28 * 3)),
    layers.Dense(len(unique_phones), activation='softmax')  # Use the corrected number of unique phones
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=64)

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

In [ ]:
model.save('drive/MyDrive/data/LSTM_01.h5')

In [ ]:

model = load_model('drive/MyDrive/data/LSTM_01.h5')

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

predictions = model.predict(X_test)

predicted_class = np.argmax(predictions[0])
print(f"Predicted class: {predicted_class}")


In [ ]:

test_image = X_test[0]
test_image = np.expand_dims(test_image, axis=0)

prediction = model.predict(test_image)
predicted_class = np.argmax(prediction)

predicted_phoneme = label_encoder.inverse_transform([predicted_class])[0]

print(f"Predicted phoneme: {predicted_phoneme}")

test_image_2d = test_image.reshape((28, 28*3))
plt.imshow(test_image_2d, cmap='gray')
plt.show()

In [ ]:


model = load_model('drive/MyDrive/data/LSTM_01.h5')

def preprocess_image(image_path, target_size=(28, 28)):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=-1)
    img = img.astype('float32') / 255.0
    return img

test_image_path = 'cropped/017462.jpg'
test_image = preprocess_image(test_image_path)

test_image = test_image.reshape((1, 28, 28 * 3))

prediction = model.predict(test_image)
predicted_class = np.argmax(prediction)

predicted_phoneme = label_encoder.inverse_transform([predicted_class])[0]

print(f"Predicted phoneme: {predicted_phoneme}")

test_image_2d = test_image.reshape((28, 28*3))
plt.imshow(test_image_2d, cmap='gray')
plt.show()


In [ ]:
data = pd.read_csv('drive/MyDrive/data/data.csv')

actual_label = data[data['image'].str.contains(test_image_path.split('/')[-1], na=False)]['phone'].values[0]
print(f"Actual label: {actual_label}")
